In [1]:
data_var = '2024-02-03'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13131,2024-02-03,Eua Ncaa,21:00,Miss. Valley St.,Arkansas-Pine Bluff,4.91,1.19,147.5,1.80,1.95,8.5,2.10,1.67,dMGryxsa,0.203666,0.840336,0.555556,0.512821,0.044002,629.092,335.615462,0.533492,0.0,0.000000,NaN,918.00,11.276,6.507137,0.577078,-91.0,226.054,80.395038,0.355645,2.4,1.341641,0.559017,282.94,2.982,1.329011,0.445678,11.0,54,86,17.00,3.29,0.000,440.948,0.862438,0.056569,0.161303,0.00,0.000,inf,0.000000,0.0,0.000000,2.26,0.452,0.420354,0.000000,0.0,0.000000
13132,2024-02-03,Eua Ncaa,22:00,Montana State,Idaho,1.25,4.19,142.5,1.91,1.91,-9.5,2.00,1.71,zeUI09TM,0.800000,0.238663,0.523560,0.523560,0.038663,167.508,43.882643,0.261973,2.4,1.341641,0.559017,163.80,2.090,0.377823,0.180776,31.0,321.386,227.675913,0.708419,0.0,0.000000,NaN,480.20,4.886,3.361507,0.687988,-67.0,70,70,2.34,6.86,151.468,492.448,0.764299,0.000000,0.110545,2.76,0.552,0.452899,0.000000,0.0,0.000000,-5.00,-1.000,-3.190000,0.000000,0.0,0.000000
13133,2024-02-03,Eua Ncaa,22:00,Louisville,Florida State,2.79,1.47,149.5,1.80,1.95,3.5,2.10,1.67,2eqBrSQl,0.358423,0.680272,0.555556,0.512821,0.038695,607.432,283.262656,0.466328,0.0,0.000000,NaN,595.20,9.166,3.736145,0.407609,-78.0,197.404,66.151343,0.335106,1.8,1.643168,0.912871,229.84,2.632,0.719632,0.273416,16.0,64,68,9.30,3.38,393.630,235.272,0.438207,0.056569,0.161303,-5.00,-1.000,-1.790000,0.383900,0.3,-0.083900,2.68,0.536,0.876866,0.000000,0.0,0.000000
13134,2024-02-03,Eua Ncaa,22:00,La Lafayette,South Alabama,1.25,4.19,149.5,1.80,1.95,-9.5,2.00,1.71,A3iQy2Ms,0.800000,0.238663,0.555556,0.512821,0.038663,134.868,44.659945,0.331138,2.4,1.341641,0.559017,86.40,1.702,0.487052,0.286165,49.0,168.496,78.782488,0.467563,0.6,1.341641,2.236068,137.28,2.278,0.908829,0.398959,-32.0,80,66,1.08,2.08,115.210,239.436,0.764299,0.056569,0.110545,0.32,0.064,3.906250,0.000000,0.0,0.000000,-2.21,-0.442,-7.217195,0.338269,0.3,-0.038269
13135,2024-02-03,Eua Ncaa,22:00,CS Northridge,UC Riverside,1.80,2.05,145.5,1.80,1.95,-2.5,2.00,1.71,IiopFmRI,0.555556,0.487805,0.555556,0.512821,0.043360,258.538,208.093649,0.804886,0.0,0.000000,NaN,334.80,3.888,2.849889,0.732996,-81.0,154.760,63.805873,0.412289,0.6,1.341641,2.236068,106.00,2.516,1.055192,0.419393,-25.0,62,53,5.40,2.00,121.810,249.358,0.091832,0.056569,0.110545,0.04,0.008,100.000000,0.000000,0.0,0.000000,-5.00,-1.000,-1.050000,0.248430,0.0,-0.248430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13419,2024-02-03,Eua Ncaa,16:00,Illinois State,Bradley,3.24,1.37,132.5,1.80,1.95,5.5,2.00,1.71,n7kfPoy0,0.308642,0.729927,0.555556,0.512821,0.038569,173.342,56.045705,0.323324,1.8,1.643168,0.912871,197.03,2.760,0.925122,0.335189,-4.0,149.376,95.464937,0.639092,2.4,1.341641,0.559017,116.45,1.822,1.075951,0.590533,39.0,61,85,3.23,1.37,127.012,161.936,0.573661,0.056569,0.110545,-2.12,-0.424,-5.283019,0.594756,0.6,0.005244,-0.49,-0.098,-3.775510,0.551900,0.6,0.048100
13420,2024-02-03,Eua Ncaa,16:00,Illinois (Chi.),Southern Illinois,2.14,1.76,134.5,1.91,1.91,1.5,1.95,1.77,Kd5FWTrQ,0.467290,0.568182,0.523560,0.523560,0.035472,208.262,169.503850,0.813897,0.0,0.000000,NaN,120.00,2.880,1.8662

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
13182,19:15,Eua Ncaa,Austin Peay,North Florida,1.83,Back Home
13183,19:15,Eua Ncaa,Texas State,Southern Miss,1.76,Back Home
13224,13:30,Suíça Liga Sb,Lausanne Basket,Lugano,1.74,Back Home
13251,14:30,França Lnb,Le Portel,Strasbourg,2.17,Back Home
13266,16:30,Itália Série A1 Feminina,Ragusa F,Campobasso F,1.56,Back Home
13285,20:00,Chile Lnb,Las Animas,Espanol Osorno,2.06,Back Home
13376,18:00,Eua Ncaa,Texas A&M,Florida,1.69,Back Home
13378,18:00,Eua Ncaa,South Carolina St,Morgan State,1.56,Back Home
13399,14:00,Eua Ncaa,Miami,Virginia Tech,1.63,Back Home
13409,16:00,Eua Ncaa,TCU,Texas,1.55,Back Home
